Import default tools

In [2]:
import numpy as np
import pandas as pd

Import requests, BeautifulSoup

In [3]:
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'html.parser')
#print(soup.prettify())

Find table

In [4]:
My_table = soup.find('table',{'class':'wikitable sortable'})
#My_table

Convert string to dataframe

In [63]:
lt = pd.read_html(str(My_table))
df = lt[0]
df = df.rename(columns={"Neighbourhood": "Neighborhood"})
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [64]:
df1 = df[df.Borough != 'Not assigned']

There is no table 'Not assigned' in Neighbourhood column

In [67]:
df1[df1.Neighborhood == 'Not assigned']

,Postal Code,Borough,Neighborhood


In [68]:
df1.shape

(103, 3)

Download csv file

In [69]:
url = "http://cocl.us/Geospatial_data"
geodf = pd.read_csv(url)
geodf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Join latlon dataframe to neighbourhood dataframe

In [70]:
df2 = df1.join(geodf.set_index('Postal Code'), on='Postal Code')

In [107]:
df3 = df2[df2['Borough'].str.contains('Toronto')]
nearby_data = df2[df2['Borough'].str.contains('Toronto')]
nearby_data.head()
#df3.iloc[1,2], df3.iloc[1,3],df3.iloc[1,4]

,Postal Code,Borough,Neighborhood,Latitude,Longitude
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
22,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
30,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [111]:
df3.iloc[0,1], df3.iloc[0,2], df3.iloc[0,3],df3.iloc[0,4]

('Downtown Toronto', 'Regent Park, Harbourfront', 43.6542599, -79.3606359)

In [120]:
#!pip install folium
import folium # map rendering library
import matplotlib.cm as cm
import colour

In [12]:
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [38]:
# type your answer here
CLIENT_ID = 'UFMC4P3J35E05A0KYJJW0KR455IBIXSCHHSZRICLJZGOZ4FW' # your Foursquare ID
CLIENT_SECRET = 'QIBETEZDQHSLZMJ2DGBQHCB01Q5VCKXDE4YQYSW0BNWVVKCM' # your Foursquare Secret
ACCESS_TOKEN = 'MQX3OFQXNV5VAIUQFVIV0KWD3E05APR02HBF3PK4JQEKH11B' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#address = '102 North End Ave, New York, NY'
#geolocator = Nominatim(user_agent="foursquare_agent")
#location = geolocator.geocode(address)
latitude = nearby_data.iloc[0,3]
longitude = nearby_data.iloc[0,4]
#print(latitude, longitude)
search_query = ''
radius = 500
#url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url


Your credentails:
CLIENT_ID: UFMC4P3J35E05A0KYJJW0KR455IBIXSCHHSZRICLJZGOZ4FW
CLIENT_SECRET:QIBETEZDQHSLZMJ2DGBQHCB01Q5VCKXDE4YQYSW0BNWVVKCM


'https://api.foursquare.com/v2/venues/explore?client_id=UFMC4P3J35E05A0KYJJW0KR455IBIXSCHHSZRICLJZGOZ4FW&client_secret=QIBETEZDQHSLZMJ2DGBQHCB01Q5VCKXDE4YQYSW0BNWVVKCM&ll=43.6623015,-79.3894938&v=20180604&radius=500&limit=100'

In [14]:
results = requests.get(url).json()

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [16]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Queen's Park,Park,43.663946,-79.392180
1,Mercatto,Italian Restaurant,43.660391,-79.387664
2,NEO COFFEE BAR,Coffee Shop,43.660130,-79.385830
3,T-Swirl Crepe,Creperie,43.663452,-79.384125
4,The Yoga Sanctuary,Yoga Studio,43.661499,-79.383636


In [17]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

32 venues were returned by Foursquare.


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
nearby_venues = getNearbyVenues(nearby_venues.name, nearby_venues.lat, nearby_venues.lng)

Queen's Park
Mercatto
NEO COFFEE BAR
T-Swirl Crepe
The Yoga Sanctuary
Tokyo Sushi
Central YMCA
Bar Volo
Como En Casa
401 Games
Coach House Restaurant
Hart House Theatre
Burrito Bandidos
Sushi Garden
Flock Rotisserie + Greens
Buddies In Bad Times Theatre
Tokyo Grill
Starbucks
Booster Juice
RBC Royal Bank
Subway
Aroma Espresso Bar
Tim Hortons
Starbucks
Hart House Gym
Tim Hortons
SUDS
Convocation Hall
Tim Hortons
Understudy Café at Gerstein
The Green Beet Cafe
Arbor Room


In [20]:
print(nearby_venues.shape)
nearby_venues.head()

(1787, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Queen's Park,43.663946,-79.39218,Queen's Park,43.663946,-79.392180,Park
1,Queen's Park,43.663946,-79.39218,Hart House Theatre,43.663571,-79.394616,Theater
2,Queen's Park,43.663946,-79.39218,Philosopher's Walk,43.666894,-79.395597,Park
3,Queen's Park,43.663946,-79.39218,Isabel Bader Theatre,43.667329,-79.392604,College Theater
4,Queen's Park,43.663946,-79.39218,Tokyo Sushi,43.665885,-79.386977,Sushi Restaurant


In [21]:
nearby_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
401 Games,58,58,58,58,58,58
Arbor Room,8,8,8,8,8,8
Aroma Espresso Bar,91,91,91,91,91,91
Bar Volo,72,72,72,72,72,72
Booster Juice,100,100,100,100,100,100
Buddies In Bad Times Theatre,65,65,65,65,65,65
Burrito Bandidos,60,60,60,60,60,60
Central YMCA,55,55,55,55,55,55
Coach House Restaurant,66,66,66,66,66,66


In [22]:
print('There are {} uniques categories.'.format(len(nearby_venues['Venue Category'].unique())))

There are 107 uniques categories.


In [119]:
# one hot encoding
nearby_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nearby_onehot['Neighborhood'] = nearby_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [nearby_onehot.columns[-1]] + list(nearby_onehot.columns[:-1])
nearby_onehot = nearby_onehot[fixed_columns]

nearby_onehot.head()

,Neighborhood,Adult Boutique,Art Museum,Arts & Crafts Store,Bank,Bar,Beer Bar,Bike Rental / Bike Share,Bookstore,Breakfast Spot,...,Strip Club,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Vegetarian / Vegan Restaurant,Video Store,Yoga Studio
0,Queen's Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Queen's Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,Queen's Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Queen's Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Queen's Park,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [24]:
nearby_onehot.shape

(1787, 108)

In [25]:
nearby_grouped = nearby_onehot.groupby('Neighborhood').mean().reset_index()
nearby_grouped

,Neighborhood,Adult Boutique,Art Museum,Arts & Crafts Store,Bank,Bar,Beer Bar,Bike Rental / Bike Share,Bookstore,Breakfast Spot,...,Strip Club,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Vegetarian / Vegan Restaurant,Video Store,Yoga Studio
0,401 Games,0.017241,0.000000,0.000000,0.000000,0.017241,0.017241,0.000000,0.017241,0.017241,...,0.017241,0.017241,0.034483,0.000000,0.017241,0.000000,0.017241,0.000000,0.000000,0.017241
1,Arbor Room,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000
2,Aroma Espresso Bar,0.000000,0.000000,0.000000,0.032967,0.021978,0.000000,0.000000,0.010989,0.000000,...,0.000000,0.000000,0.021978,0.010989,0.021978,0.000000,0.000000,0.021978,0.000000,0.000000
3,Bar Volo,0.013889,0.000000,0.000000,0.000000,0.000000,0.013889,0.000000,0.027778,0.013889,...,0.013889,0.000000,0.055556,0.000000,0.013889,0.000000,0.013889,0.000000,0.013889,0.027778
4,Booster Juice,0.000000,0.000000,0.000000,0.040000,0.010000,0.000000,0.000000,0.010000,0.000000,...,0.000000,0.000000,0.040000,0.020000,0.030000,0.000000,0.000000,0.010000,0.000000,0.000000
5,Buddies In Bad Times Theatre,0.015385,0.000000,0.000000,0.000000,0.015385,0.015385,0.000000,0.015385,0.015385,...,0.015385,0.015385,0.030769,0.000000,0.015385,0.015385,0.015385,0.000000,0.000000,0.015385
6,Burrito Bandidos,0.016667,0.000000,0.000000,0.000000,0.016667,0.016667,0.000000,0.016667,0.016667,...,0.016667,0.016667,0.033333,0.000000,0.016667,0.000000,0.016667,0.000000,0.000000,0.016667
7,Central YMCA,0.018182,0.000000,0.000000,0.000000,0.018182,0.018182,0.000000,0.018182,0.000000,...,0.018182,0.018182,0.036364,0.000000,0.018182,0.000000,0.000000,0.000000,0.000000,0.018182
8,Coach House Restaurant,0.015152,0.000000,0.000000,0.000000,0.000000,0.015152,0.000000,0.015152,0.015152,...,0.015152,0.000000,0.045455,0.000000,0.015152,0.000000,0.015152,0.000000,0.000000,0.030303
9,Como En Casa,0.014286,0.000000,0.000000,0.000000,0.000000,0.014286,0.000000,0.014286,0.014286,...,0.014286,0.000000,0.057143,0.000000,0.014286,0.000000,0.014286,0.000000,0.000000,0.028571


In [26]:
nearby_grouped.shape

(29, 108)

In [27]:
num_top_venues = 5

for hood in nearby_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = nearby_grouped[nearby_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----401 Games----
                 venue  freq
0          Coffee Shop  0.07
1              Gay Bar  0.05
2  Japanese Restaurant  0.05
3     Sushi Restaurant  0.03
4                 Park  0.03


----Arbor Room----
             venue  freq
0             Park  0.25
1             Café  0.25
2          Theater  0.12
3     Concert Hall  0.12
4  College Theater  0.12


----Aroma Espresso Bar----
                 venue  freq
0          Coffee Shop  0.21
1                 Café  0.10
2       Sandwich Place  0.09
3  Japanese Restaurant  0.04
4                 Bank  0.03


----Bar Volo----
                 venue  freq
0  Japanese Restaurant  0.07
1     Sushi Restaurant  0.06
2              Gay Bar  0.04
3          Coffee Shop  0.04
4                 Café  0.04


----Booster Juice----
                venue  freq
0         Coffee Shop  0.22
1      Sandwich Place  0.07
2                Café  0.07
3  Chinese Restaurant  0.04
4    Sushi Restaurant  0.04


----Buddies In Bad Times Theatre----
          

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [112]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = nearby_grouped['Neighborhood']

for ind in np.arange(nearby_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nearby_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,401 Games,Coffee Shop,Gay Bar,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Park,Dance Studio,Yoga Studio,Gym,Historic Site
1,Arbor Room,Park,Café,Theater,Concert Hall,College Theater,College Arts Building,Gourmet Shop,Escape Room,Department Store,Dessert Shop
2,Aroma Espresso Bar,Coffee Shop,Café,Sandwich Place,Japanese Restaurant,Italian Restaurant,Bank,Indian Restaurant,Middle Eastern Restaurant,Chinese Restaurant,Pharmacy
3,Bar Volo,Japanese Restaurant,Sushi Restaurant,Café,Gay Bar,Restaurant,Coffee Shop,Yoga Studio,Hotel,Dance Studio,Mediterranean Restaurant
4,Booster Juice,Coffee Shop,Sandwich Place,Café,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop,Sushi Restaurant,Bank,Ice Cream Shop,Italian Restaurant


In [113]:
# set number of clusters
kclusters = 5

nearby_grouped_clustering = nearby_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nearby_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 0, 1, 0, 1, 1, 1, 1, 1], dtype=int32)

In [114]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

nearby_merged = nearby_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
nearby_merged = nearby_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

nearby_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,M4E,East Toronto,The Beaches,43.676357,-79.293031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,401 Games,Coffee Shop,Gay Bar,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Park,Dance Studio,Yoga Studio,Gym,Historic Site
1,2,Arbor Room,Park,Café,Theater,Concert Hall,College Theater,College Arts Building,Gourmet Shop,Escape Room,Department Store,Dessert Shop
2,0,Aroma Espresso Bar,Coffee Shop,Café,Sandwich Place,Japanese Restaurant,Italian Restaurant,Bank,Indian Restaurant,Middle Eastern Restaurant,Chinese Restaurant,Pharmacy
3,1,Bar Volo,Japanese Restaurant,Sushi Restaurant,Café,Gay Bar,Restaurant,Coffee Shop,Yoga Studio,Hotel,Dance Studio,Mediterranean Restaurant
4,0,Booster Juice,Coffee Shop,Sandwich Place,Café,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop,Sushi Restaurant,Bank,Ice Cream Shop,Italian Restaurant
5,1,Buddies In Bad Times Theatre,Coffee Shop,Japanese Restaurant,Gay Bar,Italian Restaurant,Ramen Restaurant,Diner,Sushi Restaurant,Park,Dance Studio,Historic Site
6,1,Burrito Bandidos,Coffee Shop,Japanese Restaurant,Gay Bar,Italian Restaurant,Ramen Restaurant,Dance Studio,Park,Sushi Restaurant,Hobby Shop,Historic Site
7,1,Central YMCA,Coffee Shop,Japanese Restaurant,Gay Bar,Italian Restaurant,Sushi Restaurant,Dance Studio,Yoga Studio,Ethiopian Restaurant,Martial Arts School,Liquor Store
8,1,Coach House Restaurant,Coffee Shop,Japanese Restaurant,Café,Gay Bar,Restaurant,Sushi Restaurant,Hotel,Dance Studio,Yoga Studio,Italian Restaurant
9,1,Como En Casa,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Café,Yoga Studio,Mediterranean Restaurant,Hotel,Dance Studio


In [105]:
nearby_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,M4E,East Toronto,The Beaches,43.676357,-79.293031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,M6G,Downtown Toronto,Christie,43.669542,-79.422564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colour.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nearby_merged['Latitude'], nearby_merged['Longitude'], nearby_merged['Neighborhood'], nearby_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

ValueError: cannot convert float NaN to integer

In [94]:
#!pip install colour